# Instalaciones

In [ ]:
!pip install tflearn

!echo "Done!"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done!


# Importación de datos desde drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Librerías a emplear

In [ ]:
# NLP
import nltk
nltk.download('punkt')

# Tokenización
from nltk.tokenize import word_tokenize

#Derivación regresiva para textos en español
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

# Tensorflow
import numpy as np
import tensorflow as tf
import tflearn
import random

!echo "Done!"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
Done!


In [ ]:
print("Tensorflow version: " + tf.__version__)
print("Numpy version: " +np.__version__)

Tensorflow version: 2.8.2
Numpy version: 1.21.6


In [ ]:
# importamos el documento JSON con las intents de nuestro chatbot
import json
with open("/content/drive/MyDrive/data_bot/intenciones.json") as json_data:
    intents = json.load(json_data)

!echo "Done!"

Done!


# Desarrollo del modelo

In [ ]:
words = []
etiquetas = []
docs_x = []
docs_y = []
ignore_words = ["?", "¿", "!", "¡"]

# bucle a través de cada frase en nuestros patrones de intenciones
for intent in intents['intents']:
    for pattern in intent['patterns']:
      # tokenizamos cada palabra en la frase
        wrds = nltk.word_tokenize(pattern, "spanish")
        wrds = [word.lower() for word in wrds if word.isalpha()] # Eliminar los signos de puntuación
        # añadir a nuestra lista de palabras
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in etiquetas:
        etiquetas.append(intent['tag'])

# derivación regresiva, covertir a minúscula y eliminar los duplicados
words = [spanish_stemmer.stem(w) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# eliminar duplicados de las etiquetas
etiquetas = sorted(etiquetas)

#etiquetas = sorted(list(set(etiquetas)))
#print(etiquetas)

print (len(docs_x), "documents_x")
print (len(docs_y), "documents_y")
print (len(etiquetas), "classes", etiquetas)
print (len(words), "unique stemmed words", words)

284 documents_x
284 documents_y
31 classes ['ade_economia_finanzas_marketing_rrhh', 'antropología_sociologia', 'arqueología', 'arquitectura_edificacion_civil', 'audiovisual', 'bellas_artes', 'biologia_bioquimica_biotecnologia_alimentos', 'ciencias_ambientales', 'derecho_criminologia', 'enfermeria_farmacia_medicina_nutricion_odontologia', 'estadística', 'estudios_arabes_franceses_ingleses', 'filologia_clasica', 'filologia_hispanica', 'filosofia', 'fisica_matematicas_quimica', 'fisioterapia', 'geologia_geografía', 'historia_arte', 'historia_musica', 'inef', 'informacion_documentacion', 'logopedia', 'magisterio', 'politicas', 'psicologia', 'telecomunicaciones_informatica_electronica', 'trabajo_social', 'traduccion', 'turismo', 'óptica']
262 unique stemmed words ['a', 'air', 'algebr', 'aliment', 'ambient', 'analisis', 'analiz', 'anatom', 'animal', 'antropolog', 'aprendizaj', 'arab', 'archiv', 'argument', 'arte', 'artisit', 'astrofis', 'atencion', 'atlet', 'audiovisual', 'audit', 'autor', '

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(etiquetas))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [spanish_stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[etiquetas.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

train_x = np.array(training) # np.array(training, dtype=object) --> usar en próximas versiones para que no salte aviso en consola
train_y = np.array(output) # np.array(output, dtype=object) --> usar en próximas versiones para que no salte aviso en consola

In [ ]:
# restablecer los datos del gráfico
tf.compat.v1.reset_default_graph()

# Construir la red neuronal
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Definir el modelo y configurar el tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Iniciar el entrenamiento (aplicar el algoritmo de descenso de gradiente)
model.fit(train_x, train_y, n_epoch=2000, batch_size=8, show_metric=True)
model.save('model.tflearn_prediction')

Training Step: 71999  | total loss: 0.21721 | time: 0.150s
| Adam | epoch: 2000 | loss: 0.21721 - acc: 0.8321 -- iter: 280/284
Training Step: 72000  | total loss: 0.20440 | time: 0.153s
| Adam | epoch: 2000 | loss: 0.20440 - acc: 0.8363 -- iter: 284/284
--
INFO:tensorflow:/content/model.tflearn_prediction is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def clean_up_sentence(sentence):
    # tokenizar el patrón
    sentence_words = nltk.word_tokenize(sentence, "spanish")
    sentence_words = [word.lower() for word in sentence_words if word.isalpha()] # Eliminar los signos de puntuación
    # stem each word
    sentence_words = [spanish_stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# devuelve el array de palabras de la bolsa: 0 o 1 para cada palabra de la bolsa que existe en la frase --> Función para la elaboración de la bolsa de palabras
def bow(sentence, words, show_details=False):
    # tokenizar el patrón
    sentence_words = clean_up_sentence(sentence)
    # bolsa de palabras
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
p = bow("Se me dan bien las matemáticas", words)
print (p)
print (etiquetas)
sentence_words = nltk.word_tokenize("Esta es una frase de prueba")
sentence_words = [spanish_stemmer.stem(word.lower()) for word in sentence_words]
print(sentence_words)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
['ade_economia_finanzas_marketing_rrhh', 'antropología_sociologia', 'arqueología', 'arquitectura_edificacion_civil', 'audiovisual', 'bellas_artes', 'biologia_bioquimica_biotecnologia_alimentos', 'ciencias_ambientales', 'derecho_criminologia', 'enfermeria_farmacia_medicina_nutricion_odontologia', 'estadística', 'estudios_arabes_franceses_ingleses', 'filologia_clasica', 'filologia_hispanica', 'filosofia', 'fisica_matematicas_quimica', 'fisioterapia', 'geologia_geog

In [ ]:
print(model.predict([p]))

[[1.9606814e-01 7.8362831e-08 7.6509319e-04 1.9385633e-01 2.1363989e-05
  8.6579967e-04 1.9350294e-07 1.4271987e-05 2.0135983e-04 2.0906444e-05
  3.9675655e-03 4.2152728e-04 2.0499444e-08 2.4142886e-05 2.6294613e-06
  1.9013284e-01 3.9614709e-03 1.2947840e-04 1.5376360e-03 2.9698350e-07
  3.8362617e-04 2.8383656e-06 1.0497672e-07 1.4940513e-06 3.0012453e-07
  6.2711244e-08 2.1943998e-01 9.9364006e-05 1.6696786e-05 6.2122970e-04
  1.8744314e-01]]


In [ ]:
# guardar todas nuestras estructuras de datos
import pickle
pickle.dump( {'words':words, 'classes':etiquetas, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [ ]:
#ERROR_THRESHOLD = 0.25 # comentar esta línea y la del if ERROR para que no se criben las predicciones y modificamos los corchetes de sitio
def classify(sentence):
    # generar probabilidades a partir del modelo
    results = model.predict([bow(sentence, words)])[0]
    # filtrar las predicciones por debajo de un umbral
    results = [[i,r] for i,r in enumerate(results)] #if r>ERROR_THRESHOLD]
    # ordenar por fuerza de la probabilidad
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((etiquetas[r[0]], r[1]))
    # devuelve una tupla de intención y probabilidad
    return return_list

# Pruebas con el modelo

In [ ]:
test11 = "me gusta el viaje"
classify(test11)

[('turismo', 0.49786186),
 ('arqueología', 0.49675584),
 ('ade_economia_finanzas_marketing_rrhh', 0.0027931114),
 ('traduccion', 0.0011575242),
 ('estadística', 0.0011353522),
 ('filosofia', 8.444285e-05),
 ('derecho_criminologia', 6.7945235e-05),
 ('audiovisual', 4.7678688e-05),
 ('telecomunicaciones_informatica_electronica', 4.7574355e-05),
 ('filologia_hispanica', 3.835155e-05),
 ('ciencias_ambientales', 6.119289e-06),
 ('arquitectura_edificacion_civil', 3.2784164e-06),
 ('geologia_geografía', 7.6802905e-07),
 ('fisica_matematicas_quimica', 1.505831e-07),
 ('historia_arte', 3.191091e-08),
 ('óptica', 3.9060626e-09),
 ('bellas_artes', 2.4365874e-09),
 ('logopedia', 1.1771515e-09),
 ('filologia_clasica', 4.196346e-12),
 ('politicas', 3.03512e-12),
 ('estudios_arabes_franceses_ingleses', 4.4975568e-13),
 ('trabajo_social', 1.708245e-14),
 ('antropología_sociologia', 3.663157e-15),
 ('inef', 1.8827639e-15),
 ('fisioterapia', 3.293391e-16),
 ('psicologia', 1.3945042e-16),
 ('magisterio',

In [ ]:
entrada = input("Escribe: ")
frases = entrada.split('.')

print(frases)

Escribe: Soy una persona tranquila, que le encanta leer libros sobre historia. Me gusta enseñar a los demás. Me encanta ir al museo y viajar. El mundo árabe me fascina y la mitología griega también. Me gustaría hablar varios idiomas y soy muy empático con los demás
['Soy una persona tranquila, que le encanta leer libros sobre historia', ' Me gusta enseñar a los demás', ' Me encanta ir al museo y viajar', ' El mundo árabe me fascina y la mitología griega también', ' Me gustaría hablar varios idiomas y soy muy empático con los demás']


In [ ]:
frases_nuevas = []
for frase in frases:
  if "," in frase:
    frase_nueva = frase.split(",")
    #frases_nuevas.append(frase_nueva)
    for x in range(len(frase_nueva)):
      frases_nuevas.append(frase_nueva[x])
  else:
    frases_nuevas.append(frase)

print(frases_nuevas)

['Soy una persona tranquila', ' que le encanta leer libros sobre historia', ' Me gusta enseñar a los demás', ' Me encanta ir al museo y viajar', ' El mundo árabe me fascina y la mitología griega también', ' Me gustaría hablar varios idiomas y soy muy empático con los demás']


In [ ]:
for frase in frases_nuevas:
  print(classify(frase))

[('estudios_arabes_franceses_ingleses', 0.9480503), ('biologia_bioquimica_biotecnologia_alimentos', 0.010997555), ('antropología_sociologia', 0.010466624), ('psicologia', 0.009035637), ('politicas', 0.008063574), ('enfermeria_farmacia_medicina_nutricion_odontologia', 0.0054270625), ('ade_economia_finanzas_marketing_rrhh', 0.0012518035), ('historia_musica', 0.0011810221), ('logopedia', 0.0011374558), ('fisioterapia', 0.0010832108), ('trabajo_social', 0.0010115459), ('geologia_geografía', 0.0008992762), ('audiovisual', 0.00042863865), ('magisterio', 0.00028529754), ('filologia_hispanica', 0.00012260576), ('traduccion', 9.9440505e-05), ('turismo', 7.0123315e-05), ('ciencias_ambientales', 6.8604946e-05), ('filologia_clasica', 6.8482055e-05), ('inef', 6.222418e-05), ('historia_arte', 4.571339e-05), ('informacion_documentacion', 3.7215796e-05), ('óptica', 2.9672208e-05), ('arquitectura_edificacion_civil', 2.6052965e-05), ('telecomunicaciones_informatica_electronica', 1.970497e-05), ('fisica_

In [ ]:
classify(entrada)

[('traduccion', 0.9435446),
 ('logopedia', 0.056455407),
 ('audiovisual', 1.1737578e-09),
 ('turismo', 1.5785104e-11),
 ('filologia_hispanica', 1.9065876e-22),
 ('filologia_clasica', 1.4237737e-23),
 ('politicas', 2.0222415e-29),
 ('ade_economia_finanzas_marketing_rrhh', 0.0),
 ('antropología_sociologia', 0.0),
 ('arqueología', 0.0),
 ('arquitectura_edificacion_civil', 0.0),
 ('bellas_artes', 0.0),
 ('biologia_bioquimica_biotecnologia_alimentos', 0.0),
 ('ciencias_ambientales', 0.0),
 ('derecho_criminologia', 0.0),
 ('enfermeria_farmacia_medicina_nutricion_odontologia', 0.0),
 ('estadística', 0.0),
 ('estudios_arabes_franceses_ingleses', 0.0),
 ('filosofia', 0.0),
 ('fisica_matematicas_quimica', 0.0),
 ('fisioterapia', 0.0),
 ('geologia_geografía', 0.0),
 ('historia_arte', 0.0),
 ('historia_musica', 0.0),
 ('inef', 0.0),
 ('informacion_documentacion', 0.0),
 ('magisterio', 0.0),
 ('psicologia', 0.0),
 ('telecomunicaciones_informatica_electronica', 0.0),
 ('trabajo_social', 0.0),
 ('ópt

# Guardado

In [ ]:
from keras.models import load_model
model = model.load("model.tflearn_prediction")

INFO:tensorflow:Restoring parameters from /content/model.tflearn_prediction


# Inicio de pruebas para la implementación del análisis de sentimiento

No se llegó a usar en el modelo, por lo que representa un trabajo a futuro. De esta forma, se quiere conseguir que cuando el usuario introduzca una oración de carácter negativo, esta sea asociada con la antítesis de los resultados de la clasificación del modelo.

In [ ]:
!pip install pysentimiento

!echo "Done!"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 18.0 MB/s 
     |████████████████████████████████| 312 kB 52.1 MB/s 
     |████████████████████████████████| 175 kB 59.5 MB/s 
     |████████████████████████████████| 140 kB 54.9 MB/s 
     |████████████████████████████████| 1.1 MB 69.0 MB/s 
     |████████████████████████████████| 212 kB 33.1 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 61.4 MB/s 
     |████████████████████████████████| 127 kB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 38.9 MB/s 
     |████████████████████████████████| 144 kB 40.3 MB/s 
     |████████████████████████████████| 94 kB 4.8 MB/s 
     |████████████████████████████████| 271 kB 64.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=28111e5e7acc1dddda9f5cc246e837eb8eb46256105c16891367f45c3fe8a5e3


Done!


In [ ]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

In [ ]:
analyzer.predict("Ezio Auditore es el mejor protagonista de la saga Assassins Creed")

AnalyzerOutput(output=POS, probas={POS: 0.994, NEG: 0.003, NEU: 0.003})

In [ ]:
def classify_NEG(sentence):
    # generar probabilidades a partir del modelo
    results = model.predict([bow(sentence, words)])[0]
    # filtrar las predicciones por debajo de un umbral
    results = [[i,r] for i,r in enumerate(results)] #if r>ERROR_THRESHOLD
    # ordenar por fuerza de la probabilidad
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((etiquetas[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list[-1] # Aquí cogemos la predicción con el valor más bajo, debido a que se trata de una frase con sentimiento negativo, por lo que tiene más sentido coger las antípodas de lo obtenido en la predicción. Así no tenemos que añadir a nuestro corpus json frases en negativo, lo cual produciría confusión en la predicción y ensuciar el propio corpus.

In [ ]:
for frase in frases:
  #print(classify(frase))
  if analyzer.predict(frase).output == "NEG":
    print(classify_NEG(frase))

('magisterio', 4.9158198e-06)
